In [9]:
%load_ext autoreload
%autoreload 2

W0930 11:16:52.476318 139869702965056 word.py:92] Falling back to default tokenizer, the NLTK's `TreebankWordTokenizer()`.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from models.tokenization import *
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from models.model_architecture import *
from models.train_eval import *
from tqdm import tqdm, tqdm_notebook
parent_path='../Data/New_Data_15-06-2020/'

/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/punyajoy/.conda/envs/puny

In [2]:
annotated_df=pd.read_pickle(parent_path+'Fearspeech_data_final.pkl')


In [ ]:
torch.cuda.set_device(1)

In [3]:
params={'model_path':'bert-base-multilingual-cased',
        'preprocess_doc':False,
        'max_length':256,
        'batch_size':32,
        'hidden_size':128,
        'weights':[1.0,1.0],
        'seq_model':'lstm',
        'data_path':parent_path+'Fearspeech_data_final.pkl',
        'max_sentences_per_doc':5,
        'transformer_type':'lstm_transformer',
        'device':'cpu',
        'learning_rate':5e-5,
        'epsilon':1e-8,
        'random_seed':2020,
        'epochs':50,
        'max_memory':0.5
       }


In [4]:
train_phase(params)

  8%|▊         | 390/4782 [00:04<00:45, 97.34it/s] 

KeyboardInterrupt: 

In [6]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
    

In [ ]:
temp=tokenizer.tokenize("I am goof")



In [ ]:
temp=['<s>']+temp+['</s>']

In [ ]:
tokenizer.convert_tokens_to_ids(temp)

In [27]:
from transformers import AutoConfig
config=AutoConfig.from_pretrained("xlm-roberta-base")

In [28]:
config

XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 250002
}

In [8]:
annotated_df=pd.read_pickle(params['data_path'])
params_preprocess={'remove_numbers': True, 'remove_emoji': True, 'remove_stop_words': False, 'tokenize': False}
list_sents=[preprocess_sent(ele,params=params_preprocess).strip() for ele in tqdm_notebook(annotated_df['message_text'],total=len(annotated_df))]

  8%|▊         | 393/4782 [03:27<38:40,  1.89it/s]
/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


#### Getting random samples of preprocessed text

In [20]:
import random 
random.sample(list_sents,5)

['बाबरी मस्जिद के बाद सुप्रीम कोर्ट भारत का दूसरा बड़ा विवादित ढांचा लग रहा है एक दिन इसे भी गिराना होगा',
 'ƒγgнτ\xa0cℓυϐ के नियम यहाँ पर आप किसी भी देश द्रोह मुल्ले को लाये और उसकी g चुदाई करवा कर उसे प्रसन्न करें जिस भी मुल्ले को लगता ह उसकी चुदाई g की रफ्तार से हुई है और ओ प्रसन्न नही तो हमे इस चीज के लिए खेद रहेगा लेकिन हम आपसे वादा करते कि दुबारा चुदाई g के रफ्तार होगा और अगर हामरे इंटनेट ने हमारा साथ दे दिया तो ओ चुदाई g g के रफ्तार से भी हो सकता है हमे एक बार सेवा का मौका अवश्य दे आपका आज्ञाकारी मुल्लो का बाप जीजा ताऊपापा अब्बा आदि',
 'हिन्दु जन जाग्रति मंच जय श्रीराम यह झूठ भी हमे पढ़ाया जाता रहा है कि में देश का बंटवारा हुआ था यह सरासर झूठ है में कांग्रेस ने मुसलमानों को रहने के लिए दो राष्ट्र दिए जो चाहे भारत मे रह सकता है और जिसे ज्यादा खुजली है वह पाकिस्तान चला जाये फिर बंगाली मुसलमानों के लिए बांग्लादेश बन गया यानी मुसलमानों को राष्ट्र मिले भारत पाकिस्तान और बांग्लादेश पाकिस्तान और बांग्लादेश से बड़े ही सुनियोजित ढंग से हिंदुओं का सफाया हो गया केवल भारत मे सिमटा हिन्दू अब अ

In [9]:
count=[]
for sent in list_sents:
    count.append(len(tokenizer.tokenize(sent)))

X_0 = np.array(list_sents,dtype='object')
y_0 = np.array(annotated_df['one_fear_speech'])
#tokenizer = AutoTokenizer.from_pretrained(params['model_path'])
    
X_train = encode_documents(X_0,params,tokenizer) 

NameError: name 'document' is not defined

In [23]:
count_list=[]
for sent in list_sents:
    temp=0
    for ele in tokenizer.encode(sent,add_special_tokens=True):
        if ele==3:
            temp+=1  
    if(temp>0):
        count_list.append(temp)
    

Token indices sequence length is longer than the specified maximum sequence length for this model (818 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (895 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1519 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (704 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (784 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

Token indices sequence length is longer than the specified maximum sequence length for this model (1856 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1682 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1191 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2176 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (692 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

Token indices sequence length is longer than the specified maximum sequence length for this model (1153 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1318 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2754 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1409 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (678 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

Token indices sequence length is longer than the specified maximum sequence length for this model (990 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (646 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1652 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1964 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (744 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

Token indices sequence length is longer than the specified maximum sequence length for this model (625 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1380 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (645 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (584 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1949 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

...

Token indices sequence length is longer than the specified maximum sequence length for this model (696 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (729 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1241 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (849 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2233 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

Token indices sequence length is longer than the specified maximum sequence length for this model (1676 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1091 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1261 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1482 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1357 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

Token indices sequence length is longer than the specified maximum sequence length for this model (2687 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (836 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2753 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (792 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1017 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

Token indices sequence length is longer than the specified maximum sequence length for this model (1066 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1323 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (959 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (835 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (787 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

Token indices sequence length is longer than the specified maximum sequence length for this model (742 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (947 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (595 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (598 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (808 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [25]:
len(count_list)

70

In [22]:
tokenizer.encode('tere ma kaa',add_special_tokens=True)

[0, 71440, 291, 42190, 2]

In [15]:
tokenizer.unk_token

'<unk>'

In [13]:
count[0:5]

[['▁प्रशासक',
  '▁समिति',
  '▁',
  'आं',
  'तक',
  'वादी',
  '▁संगठन',
  'ों',
  '▁का',
  '▁इस्लाम',
  'िक',
  '▁नाम',
  '▁और',
  '▁उनका',
  '▁इस्लाम',
  '▁धर्म',
  '▁से',
  '▁जुड़',
  'ा',
  '▁हुआ',
  '▁अर्थ',
  '▁ल',
  'श्',
  'कर',
  'े',
  '▁',
  'तै',
  'य',
  'बा',
  'फ',
  'रि',
  'श्',
  'तो',
  '▁की',
  '▁सेना',
  '▁अल',
  '▁कायदा',
  'अ',
  'ल्ला',
  'ह',
  '▁का',
  '▁कायदा',
  '▁जे',
  'श',
  '▁ए',
  '▁मोह',
  'म्',
  'द',
  'मोह',
  'म्',
  'मद',
  '▁',
  'साहेब',
  '▁का',
  '▁दल',
  '▁तह',
  'रिक',
  '▁ए',
  '▁ता',
  'लि',
  'बान',
  'प',
  'वि',
  'त्र',
  '▁यो',
  'द्धा',
  'ओ',
  '▁का',
  '▁दल',
  '▁हि',
  'ज',
  'बुल',
  '▁मु',
  'जा',
  'दिन',
  'इस',
  '्ला',
  'मी',
  '▁ब',
  'लि',
  'दान',
  'ियो',
  '▁का',
  '▁समूह',
  '▁बो',
  'को',
  '▁ह',
  'राम',
  '▁पै',
  'ग',
  'म्बर',
  '▁मुह',
  'म्',
  'मद',
  '▁की',
  '▁शिक्षा',
  '▁को',
  '▁फैला',
  'ने',
  '▁के',
  '▁लिए',
  '▁प्रति',
  'बद्ध',
  '▁सभी',
  '▁मु',
  'स्',
  'ल',
  'मान',
  '▁अ',
  'ज्ञान',
  'ि',
  '▁अ

In [ ]:
count_greater=0
for ele in count:
    if(ele > (128*5)):
        count_greater+=1
count_greater

In [ ]:
count

In [ ]:
train_dataloader=return_dataloader(X_train,y_0,params,is_train=True)


In [ ]:
train_dataloader

In [ ]:
import torch
import torch.autograd.profiler as profiler
with profiler.profile(record_shapes=True) as prof:
    with profiler.record_function("model_inference"):
        for step, batch in tqdm(enumerate(train_dataloader)):
                b_input_ids = batch[0]
                b_att_masks = batch[1]
                b_token_types = batch[2]
                b_labels = batch[3]
                outputs = model(input_ids=b_input_ids, attention_mask=b_att_masks, token_type_ids=b_token_types,labels=b_labels)
                break




In [ ]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

In [ ]:
b_input_ids.shape

In [ ]:
b_input_ids[0][:20,1]

In [ ]:
model=select_transformer_model(params['transformer_type'],params['model_path'],params)

In [ ]:
import torch


In [ ]:
inputs = torch.randn((5, 3, 224, 224),dtype='long')

In [ ]:
inputs